In [2]:
# Import libriries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
import pickle
from utils import *

Using TensorFlow backend.


In [2]:
# # Export Training Data and dropout NaN rows
# EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
# df = pd.read_csv('questions.csv')
# df = df.dropna()


In [3]:
# # Data Processing
# # 1) Take unique words from questions
# results = set()
# df['question1'].str.lower().str.split().apply(results.update)
# df['question2'].str.lower().str.split().apply(results.update)
# # Now results is a bag of unique words
# # 2) Store indices of unique words
# idx2word = dict(enumerate(results))
# word2idx = {word: idx for idx, word in idx2word.items()}

In [4]:
# # Encode questions to index
# def encode_question(question):
#     words = question.split()
#     encoded = []
#     for word in words:
#         encoded.append(word2idx[word])
#     return encoded 

# df['enc_question1'] = df.question1.str.lower().apply(encode_question)
# df['enc_question2'] = df.question2.str.lower().apply(encode_question)

In [5]:

# embedding_dim = 300
# embeddings = 1 * np.random.randn(len(results) + 1, embedding_dim)  # This will be the embedding matrix
# embeddings[0] = 0  # So that the padding will be ignored

# # Build the embedding matrix
# for word, index in word2idx.items():
#     if word in word2vec.vocab:
#         embeddings[index] = word2vec.word_vec(word)

In [3]:
filename = 'embeddings'
infile = open(filename,'rb')
embeddings = pickle.load(infile)
embedding_dim = 300

filename = 'df'
infile = open(filename,'rb')
df = pickle.load(infile)


In [4]:
train_df = df
questions_cols = ['enc_question1', 'enc_question2']
max_seq_length = max(train_df.enc_question1.map(lambda x: len(x)).max(),
                     train_df.enc_question2.map(lambda x: len(x)).max())
# Split to train validation
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.enc_question1, 'right': X_train.enc_question2}
X_validation = {'left': X_validation.enc_question1, 'right': X_validation.enc_question2}
# X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [8]:
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 20

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden, activation='tanh', recurrent_activation='hard_sigmoid')

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)


# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
# Start training
training_start_time = time()

checkpointer = ModelCheckpoint(filepath='tanh_mycode/weights.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.hdf5', verbose=1, save_best_only=True)

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch, 
                            callbacks=[checkpointer], validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

ResourceExhaustedError: OOM when allocating tensor with shape[201114,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node embedding_4/random_uniform/RandomUniform (defined at /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139)  = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=1730892, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_4/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'embedding_4/random_uniform/RandomUniform', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home